<a href="https://colab.research.google.com/github/VictoriaRe/HSE-DataScience/blob/master/Assignment8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Assignment 8

Develop a model for 20 news groups dataset from scikit-learn.
Select 20% of data for test set. 

Develop metric learning model with siamese network [3 points] and triplet loss [3 points] (from seminar).


Use KNN and LSH (any library for approximate nearest neighbor search) for final prediction after the network was trained. [2 points]


! Remember, that LSH gives you a set of neighbor candidates, for which you have to calculate distances to choose top-k nearest neighbors. Your quality metric = accuracy score [2 points if acc > 0.8 ]

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
newsgroups_all = fetch_20newsgroups(subset='all')

In [0]:
import pandas
import nltk

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
data = pandas.DataFrame(columns=['text', 'categ'])
data['text'] = newsgroups_all.data
data['text']  = data['text'].apply(lambda sent: nltk.word_tokenize(sent))
data['categ'] = newsgroups_all.target

In [6]:
data.head()

,text,categ
0,"[From, :, Mamatha, Devineni, Ratnam, <, mr47+,...",10
1,"[From, :, mblawson, @, midway.ecn.uoknor.edu, ...",3
2,"[From, :, hilmi-er, @, dsv.su.se, (, Hilmi, Er...",17
3,"[From, :, guyd, @, austin.ibm.com, (, Guy, Daw...",3
4,"[From, :, Alexander, Samuel, McDiarmid, <, am2...",4


Я пробовала обучать на Элмо, модель сликшом громоздкая и даже колаб не справляется и постоянно рестартиться, поэтому беру W2V, обученный на гугл-новостях (хотя мы часто брали вики, но она не отразит специфику наших данных)

In [0]:
import gensim
import gensim.downloader as api

In [8]:
w2v = api.load('word2vec-google-news-300')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Собственно, теперь векторизуем все предложения нашего дата сета

In [0]:
import numpy

In [0]:
def vectorize(sent, w2v):

  sent2vec = []

  for word in sent:
    
    try:
      sent_vec = w2v[word]
      sent2vec.append(sent_vec)
    
    except KeyError:
      continue
  
  return numpy.mean(numpy.array(sent2vec), axis=0)

Добавим теперь это всё в наш датафрейм

In [0]:
data['sent2vec'] = data['text'].apply(lambda sent: vectorize(sent, w2v))

In [12]:
data.head()

,text,categ,sent2vec
0,"[From, :, Mamatha, Devineni, Ratnam, <, mr47+,...",10,"[0.030878138, 0.034033768, 0.014451373, 0.0887..."
1,"[From, :, mblawson, @, midway.ecn.uoknor.edu, ...",3,"[-0.0047734827, -0.0040349867, 0.021684444, 0...."
2,"[From, :, hilmi-er, @, dsv.su.se, (, Hilmi, Er...",17,"[0.012571868, 0.015153378, 0.07615096, 0.09736..."
3,"[From, :, guyd, @, austin.ibm.com, (, Guy, Daw...",3,"[-0.052938033, -0.01901048, 0.029367056, 0.086..."
4,"[From, :, Alexander, Samuel, McDiarmid, <, am2...",4,"[0.02456878, 0.0010233887, 0.032747492, 0.0793..."


In [0]:
from sklearn.datasets import fetch_20newsgroups_vectorized

In [0]:
newsgroups_all_vect = fetch_20newsgroups(subset='all')

In [15]:
newsgroups_all_vect['DESCR']

'.. _20newsgroups_dataset:\n\nThe 20 newsgroups text dataset\n------------------------------\n\nThe 20 newsgroups dataset comprises around 18000 newsgroups posts on\n20 topics split in two subsets: one for training (or development)\nand the other one for testing (or for performance evaluation). The split\nbetween the train and test set is based upon a messages posted before\nand after a specific date.\n\nThis module contains two loaders. The first one,\n:func:`sklearn.datasets.fetch_20newsgroups`,\nreturns a list of the raw texts that can be fed to text feature\nextractors such as :class:`sklearn.feature_extraction.text.CountVectorizer`\nwith custom parameters so as to extract feature vectors.\nThe second one, :func:`sklearn.datasets.fetch_20newsgroups_vectorized`,\nreturns ready-to-use features, i.e., it is not necessary to use a feature\nextractor.\n\n**Data Set Characteristics:**\n\n    =================   ==========\n    Classes                     20\n    Samples total            

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42, shuffle=True)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42, shuffle=True)

In [21]:
len(train_data)

13568

In [22]:
train_data.head()

,text,categ,sent2vec
2855,"[From, :, fragante, @, unixg.ubc.ca, (, Gv, Fr...",3,"[-0.02576294, 0.0120181525, 0.07280721, 0.0881..."
13738,"[Subject, :, Technical, Help, Sought, From, :,...",1,"[0.014058024, 0.015347494, 0.041504286, 0.0699..."
2015,"[From, :, delliott, @, access.digex.com, (, Da...",6,"[-0.008551801, -0.020215895, -0.013249757, 0.0..."
3957,"[From, :, bd, @, fluent, @, dartmouth.EDU, (, ...",8,"[-0.009589086, 0.012489801, 0.02656729, 0.0518..."
18581,"[From, :, gtonwu, @, Uz.nthu.edu.tw, (, Tony, ...",2,"[-0.00019626885, 0.030809645, 0.040813606, 0.0..."


In [0]:
def positive(row, data):

  categ = row['categ'] 
  vec = row['sent2vec']
  pos = data[data['categ'] == categ]['sent2vec']
  selec = numpy.random.choice(pos)

  while selec.tolist() == vec.tolist():
    selec = numpy.random.choice(pos)

  return selec

In [0]:
def negative(row, data):

  categ = row['categ'] 
  vec = row['sent2vec']
  negative = data[data['categ'] != categ]['sent2vec']

  return numpy.random.choice(negative)

In [0]:
pos = []
neg = []

for i, row in train_data.iterrows():

  pos.append(positive(row, train_data))
  neg.append(negative(row, train_data))

In [26]:
len(pos)

13568

In [0]:
train_data['positive'] = pos
train_data['negative'] = neg

In [28]:
train_data.head()

,text,categ,sent2vec,positive,negative
2855,"[From, :, fragante, @, unixg.ubc.ca, (, Gv, Fr...",3,"[-0.02576294, 0.0120181525, 0.07280721, 0.0881...","[0.004194855, 0.013896293, 0.020245459, 0.0965...","[0.045015533, 0.0041036936, 0.0117338775, 0.05..."
13738,"[Subject, :, Technical, Help, Sought, From, :,...",1,"[0.014058024, 0.015347494, 0.041504286, 0.0699...","[-0.00077989366, 0.017056076, 0.051457606, 0.0...","[-0.02316532, 0.019990012, 0.05647983, 0.06446..."
2015,"[From, :, delliott, @, access.digex.com, (, Da...",6,"[-0.008551801, -0.020215895, -0.013249757, 0.0...","[0.018569104, -0.017542018, 0.040247425, 0.069...","[-0.037702426, -0.0081015425, 0.028898591, 0.0..."
3957,"[From, :, bd, @, fluent, @, dartmouth.EDU, (, ...",8,"[-0.009589086, 0.012489801, 0.02656729, 0.0518...","[-0.008533771, 0.022494389, 0.025206346, 0.051...","[0.02756388, 0.014384871, 0.040134203, 0.07167..."
18581,"[From, :, gtonwu, @, Uz.nthu.edu.tw, (, Tony, ...",2,"[-0.00019626885, 0.030809645, 0.040813606, 0.0...","[-0.03536923, -0.009965627, 0.02977832, 0.1007...","[-0.017275603, 0.017655082, 0.053933848, 0.083..."


In [29]:
len(train_data)

13568

In [0]:
pos_t = []
neg_t = []

for i, row in test_data.iterrows():

  pos_t.append(positive(row, test_data))
  neg_t.append(negative(row, test_data))

In [31]:
len(pos_t)

3770

In [32]:
test_data['positive'] = pos_t
test_data['negative'] = neg_t

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
test_data.head()

,text,categ,sent2vec,positive,negative
18265,"[From, :, shippert, @, cco.caltech.edu, (, Tim...",9,"[0.007987103, 0.03350349, 0.04697369, 0.072084...","[-0.0029450874, 0.049150147, 0.04321729, 0.080...","[0.015548784, 0.010420511, 0.05369127, 0.11716..."
423,"[From, :, mont, @, netcom.com, (, Mont, Pierce...",12,"[-0.0074178474, -0.011049128, 0.02438422, 0.09...","[-0.03989665, -0.015083525, 0.026442464, 0.083...","[-0.012544071, 0.04865098, 0.03800695, 0.06475..."
7900,"[From, :, Amruth, Laxman, <, al26+, @, andrew....",14,"[0.0016954596, 0.032415245, 0.014972788, 0.079...","[-0.024588013, 0.01792805, 0.0387215, 0.064285...","[-0.0987304, -0.0023376315, 0.0538158, 0.06732..."
14785,"[From, :, kbanaian, @, bernard.pitzer.claremon...",18,"[-0.009543864, 0.010596233, 0.032779463, 0.109...","[0.03449735, 0.02204775, 0.038949084, 0.102718...","[0.007518627, 0.029511629, 0.003592032, 0.0775..."
5217,"[From, :, mcelwre, @, cnsvax.uwec.edu, Subject...",0,"[0.011646996, 0.023111023, 0.0551277, 0.055711...","[-0.011638916, -0.016002404, 0.052872546, 0.13...","[0.021784522, 0.011611854, 0.030614667, 0.0918..."


In [0]:
pos_val = []
neg_val = []

for i, row in val_data.iterrows():

  pos_val.append(positive(row, val_data))
  neg_val.append(negative(row, val_data))

In [0]:
val_data['positive'] = pos_val
val_data['negative'] = neg_val

In [0]:
batch_size = 256
random_state = 42

In [0]:
class Batch:

  def __init__(self, anchors, positives, negatives, targets):

    self.anchor = anchors
    self.positive = positives
    self.negative = negatives
    self.categ = targets

In [0]:
class Iterator:

  def __init__(self, df, batch_size = 256, device = 'cpu'):
    self.batches = self.make_batches(df, batch_size, device)

  @staticmethod
  def make_batches(df, batch_size, device):

    batches = []
    shuffled = df.sample(frac = 1)
    i = 0

    while i < len(df):
      bat4 = shuffled[i: i + batch_size]

      if len(bat4) == batch_size:

        anchor = torch.tensor(list(bat4.sent2vec.values)).to(device)
        positive = torch.tensor(list(bat4.positive.values)).to(device)
        negative = torch.tensor(list(bat4.negative.values)).to(device)
        categ = torch.tensor(list(bat4.categ.values)).to(device)

        batch = Batch(anchor, positive, negative, categ)
        batches.append(batch)

      i += batch_size

    return batches

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [0]:
train_iter = Iterator(train_data, batch_size = 256, device = DEVICE).batches

In [0]:
test_iter = Iterator(test_data, batch_size = 256, device = DEVICE).batches

In [0]:
val_iter = Iterator(val_data, batch_size = 256, device = DEVICE).batches

In [0]:
class MyModel(nn.Module):
    
    def __init__(self, inp_dim = 300, lin_dim = 128):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(inp_dim, lin_dim)
        
    def branch(self, x):
        x = self.fc(x)
        return x
        
    def forward(self, batch):
        anchor = batch.anchor
        positive = batch.positive
        negative = batch.negative
        
        anchor = self.branch(anchor)
        positive = self.branch(positive)
        negative = self.branch(negative)
        
        return anchor, positive, negative

In [0]:
def train_epoch(data_iter, len_iter, n_epoch, model, criterion, optimizer = None):

    train_losses = []
    total_loss = 0
    data_iter = tqdm_notebook(data_iter, total = len_iter, desc = f"Epoch {n_epoch + 1}", leave = True)
    counter = 0
    
    for batch in data_iter:

        if optimizer:
          optimizer.zero_grad()

        anchor, positive, negative = model.forward(batch)
        loss = criterion(anchor, positive, negative)
        loss.backward()
        
        if optimizer:
          optimizer.step()
        loss_value = loss.detach().item()
        total_loss += loss_value
        train_losses.append(loss_value)
        data_iter.set_postfix(loss = loss_value)
        counter += 1
        
    total_loss /= counter
    
    return total_loss, train_losses


def valid_epoch(data_iter, len_iter, n_epoch, model, criterion):

    valid_losses = []
    total_loss = 0
    data_iter = tqdm_notebook(data_iter, total = len_iter, desc = f"Evaluation epoch {n_epoch + 1}", leave = True)
    counter = 0
    
    for batch in data_iter:
        with torch.no_grad():
            anchor, positive, negative = model.forward(batch)
            loss = criterion(anchor, positive, negative)
            loss_value = loss.detach().item()
            total_loss += loss_value
            valid_losses.append(loss_value)
            data_iter.set_postfix(loss = loss_value)
            counter +=1
        
    total_loss /= counter
    return total_loss, valid_losses

In [0]:
class TripletLoss(nn.L1Loss):
  def __init__(self, margin = 0.1):
    
    super(TripletLoss, self).__init__()
    self.margin = margin
  
  def forward(self, anchor, positive, negative):
    return torch.mean(F.cosine_similarity(anchor, negative, dim = -1) - F.cosine_similarity(anchor, positive, dim = -1) + self.margin)

In [0]:
criterion = TripletLoss()

In [0]:
criterion = TripletLoss()
criterion = criterion.to(DEVICE)
model = MyModel(inp_dim = 300, lin_dim = 512)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience = 3)

In [0]:
num_epochs = 9

In [0]:
from tqdm import tqdm, tqdm_notebook

In [71]:
total_train_losses = []
total_valid_losses = []
for epoch in range(num_epochs):
    model.train()
    loss, train_losses = train_epoch(train_iter, len(train_data), epoch, model, criterion, optimizer)
    total_train_losses += train_losses
    print('train', loss)
    
    model.eval()
    with torch.no_grad():
        loss, valid_losses = valid_epoch(val_iter, len(val_data), epoch, model, criterion)
        total_valid_losses += valid_losses
        
        if scheduler:
          if type(scheduler) == torch.optim.lr_scheduler.ReduceLROnPlateau:
            scheduler.step(loss)
          else:
            scheduler.step() 
        
        print('valid', loss)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



train -0.3770726197838502


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



valid -0.5413598656654358



train -0.6170419707613172


...


valid -0.6879318952560425



train -0.7439787219155509



valid -0.6942057371139526



train -0.750823216618232



valid -0.6992884635925293


In [85]:
predict_vectors = []
predict_categs = []

model.eval()

with torch.no_grad():

  data_iter = tqdm_notebook(test_iter, total = len(test_data), desc = f"Test", leave = True)

  for batch in data_iter:

    anchor, positive, negative = model.forward(batch)
    categ = batch.categ
    
    predict_vectors.append(anchor.cpu().detach().numpy())
    predict_categs += categ.cpu().detach()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [0]:
predict_data = pandas.DataFrame(columns = ['category', 'sent2vec'])

predict_data['category'] = [x.item() for x in predict_categs]
predict_data['sent2vec'] = list(numpy.concatenate(predict_vecs))

In [0]:
from sklearn.neighbors import NearestNeighbors

In [0]:
X = numpy.array([[-2, -1, 2], [-3, -2, 1], [1, 2, 1], [1, 3, 2], [-1, -1, 3], [1, 1, 1]])

neighbors = NearestNeighbors(n_neighbors = 3, algorithm = 'ball_tree').fit(X)
x_distances, x_indices = neighbors.kneighbors(X)

In [0]:
predicted = numpy.concatenate(predict_vectors)
neighbors = NearestNeighbors(n_neighbors = 3, algorithm = 'ball_tree').fit(predicted)
distances, indices = neighbors.kneighbors(predicted)

In [0]:
predicted_classes = []

for i, row in predict_data.iterrows():

  distances_to_classes = [100] * 30
  distances_i = distances[i]
  indices_i = indices[i]

  for n_i, n_d in enumerate(distances_i):
    class_n = predict_data['category'][indices_i[n_i]]

    if distances_to_classes[int(class_n)] == 100:
      distances_to_classes[int(class_n)] = n_d

    else:
      distances_to_classes[int(class_n)] += n_d
  
  class_i = distances_to_classes.index(min(distances_to_classes))

  predicted_classes.append(class_i)

In [0]:
from sklearn.metrics import accuracy_score

In [118]:
accuracy_score(y_true = list(predict_data.category.values), y_pred = predicted_classes)

0.8699776785714286